In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense, Activation
from keras.regularizers import l2
from keras.optimizers import Adam
import tensorflow as tf
from keras import layers
import keras
import utils

In [6]:
DATA_PATH = "/Users/mykola/MLHEP/Wiremind/"
raw_features = pd.read_csv(os.path.join(DATA_PATH, "ds_train.csv"))

Features are introduced in the code file utils.py. 

In [7]:
all_features = utils.features_preparation(raw_features)
scaled_features = utils.features_scale(all_features)

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model
import tensorlayer as tl
main_input = Input(shape=(12,))
dense1 = (Dense(1024, activation=lambda x : tl.act.lrelu(x, 0.2), kernel_initializer='random_normal'))(main_input)
dpt1 = (layers.Dropout(rate=0.25))(dense1)
dense2 = (Dense(512, activation=lambda x : tl.act.lrelu(x, 0.2), kernel_initializer='random_normal'))(dpt1)
dpt2 = (layers.Dropout(rate=0.2))(dense2)
dense3 = (Dense(256, activation=lambda x : tl.act.lrelu(x, 0.2), kernel_initializer='random_normal'))(dpt2)
dense4 = (Dense(128, activation=lambda x : tl.act.lrelu(x, 0.2), kernel_initializer='random_normal')) (dense2)
primary_out = Dense(1,activation='linear')(dense3)
combined_input  = keras.layers.concatenate([main_input, primary_out])
dense21 = Dense(256,activation='relu')(combined_input)
bn1 = layers.BatchNormalization()(dense21)
dense22 = Dense(128,activation='relu')(bn1)
bn2 = layers.BatchNormalization()(dense22)
secondary_out = Dense(1,activation='linear')(bn2)
secondary_out.trainable = False

In [9]:
train_part, validation = train_test_split(scaled_features, test_size=0.2, shuffle=True, random_state=342343234)
print(train_part.shape)
print(validation.shape)
target = "demand"

(234120, 14)
(58530, 14)


In [249]:
model = Model(inputs=[main_input],outputs=[primary_out,secondary_out])
save_all_xy = keras.callbacks.ModelCheckpoint("NNmodels/"+'weights.{epoch:02d}-{val_loss:.5f}.h5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
model.compile(loss=["MAE"], metrics="MSE", optimizer=Adam(lr=0.001),loss_weights=[1])
history = model.fit(train_part.loc[:,utils.input_features].values,train_part.demand, verbose=1, callbacks=[save_all_xy], validation_data=(validation.loc[:,utils.input_features].values, validation.demand), shuffle=False, batch_size=150, epochs=1200)


Epoch 1/1200
1557/1561 [============================>.] - ETA: 0s - loss: 1.0232 - dense_113_loss: 1.0232 - dense_113_MSE: 7.0787 - dense_116_MSE: 13.0695
Epoch 00001: saving model to NNmodels/weights.01-0.91918.h5
1561/1561 [==============================] - 14s 9ms/step - loss: 1.0232 - dense_113_loss: 1.0232 - dense_113_MSE: 7.0741 - dense_116_MSE: 13.0705 - val_loss: 0.9192 - val_dense_113_loss: 0.9192 - val_dense_113_MSE: 5.0681 - val_dense_116_MSE: 12.1751
Epoch 2/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.9328 - dense_113_loss: 0.9328 - dense_113_MSE: 5.3946 - dense_116_MSE: 12.7363
Epoch 00002: saving model to NNmodels/weights.02-0.88338.h5
1561/1561 [==============================] - 14s 9ms/step - loss: 0.9329 - dense_113_loss: 0.9329 - dense_113_MSE: 5.3927 - dense_116_MSE: 12.7379 - val_loss: 0.8834 - val_dense_113_loss: 0.8834 - val_dense_113_MSE: 4.7024 - val_dense_116_MSE: 12.3306
Epoch 3/1200
1558/1561 [============================>.] - ETA: 0s 

Epoch 33/1200
1558/1561 [============================>.] - ETA: 0s - loss: 0.8422 - dense_113_loss: 0.8422 - dense_113_MSE: 4.1676 - dense_116_MSE: 12.5825
Epoch 00033: saving model to NNmodels/weights.33-0.81882.h5
1561/1561 [==============================] - 19s 12ms/step - loss: 0.8422 - dense_113_loss: 0.8422 - dense_113_MSE: 4.1662 - dense_116_MSE: 12.5820 - val_loss: 0.8188 - val_dense_113_loss: 0.8188 - val_dense_113_MSE: 3.7319 - val_dense_116_MSE: 12.0683
Epoch 34/1200
1560/1561 [============================>.] - ETA: 0s - loss: 0.8405 - dense_113_loss: 0.8405 - dense_113_MSE: 4.1635 - dense_116_MSE: 12.5780
Epoch 00034: saving model to NNmodels/weights.34-0.82375.h5
1561/1561 [==============================] - 19s 12ms/step - loss: 0.8405 - dense_113_loss: 0.8405 - dense_113_MSE: 4.1627 - dense_116_MSE: 12.5756 - val_loss: 0.8237 - val_dense_113_loss: 0.8237 - val_dense_113_MSE: 3.6997 - val_dense_116_MSE: 11.9678
Epoch 35/1200
1561/1561 [==============================] - ETA

Epoch 51/1200
1558/1561 [============================>.] - ETA: 0s - loss: 0.8341 - dense_113_loss: 0.8341 - dense_113_MSE: 4.0707 - dense_116_MSE: 12.5731
Epoch 00051: saving model to NNmodels/weights.51-0.81176.h5
1561/1561 [==============================] - 16s 10ms/step - loss: 0.8340 - dense_113_loss: 0.8340 - dense_113_MSE: 4.0691 - dense_116_MSE: 12.5725 - val_loss: 0.8118 - val_dense_113_loss: 0.8118 - val_dense_113_MSE: 3.6385 - val_dense_116_MSE: 12.0418
Epoch 52/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.8320 - dense_113_loss: 0.8320 - dense_113_MSE: 4.0579 - dense_116_MSE: 12.5740
Epoch 00052: saving model to NNmodels/weights.52-0.81805.h5
1561/1561 [==============================] - 16s 10ms/step - loss: 0.8320 - dense_113_loss: 0.8320 - dense_113_MSE: 4.0560 - dense_116_MSE: 12.5759 - val_loss: 0.8181 - val_dense_113_loss: 0.8181 - val_dense_113_MSE: 3.6488 - val_dense_116_MSE: 12.0307
Epoch 53/1200
1560/1561 [============================>.] - ETA

Epoch 69/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.8278 - dense_113_loss: 0.8278 - dense_113_MSE: 4.0113 - dense_116_MSE: 12.5689
Epoch 00069: saving model to NNmodels/weights.69-0.81708.h5
1561/1561 [==============================] - 17s 11ms/step - loss: 0.8278 - dense_113_loss: 0.8278 - dense_113_MSE: 4.0100 - dense_116_MSE: 12.5711 - val_loss: 0.8171 - val_dense_113_loss: 0.8171 - val_dense_113_MSE: 3.5937 - val_dense_116_MSE: 12.0481
Epoch 70/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.8277 - dense_113_loss: 0.8277 - dense_113_MSE: 4.0166 - dense_116_MSE: 12.5635
Epoch 00070: saving model to NNmodels/weights.70-0.81540.h5
1561/1561 [==============================] - 15s 10ms/step - loss: 0.8276 - dense_113_loss: 0.8276 - dense_113_MSE: 4.0140 - dense_116_MSE: 12.5654 - val_loss: 0.8154 - val_dense_113_loss: 0.8154 - val_dense_113_MSE: 3.6266 - val_dense_116_MSE: 12.0178
Epoch 71/1200
1558/1561 [============================>.] - ETA

Epoch 87/1200
1555/1561 [============================>.] - ETA: 0s - loss: 0.8248 - dense_113_loss: 0.8248 - dense_113_MSE: 3.9584 - dense_116_MSE: 12.5539
Epoch 00087: saving model to NNmodels/weights.87-0.80144.h5
1561/1561 [==============================] - 15s 10ms/step - loss: 0.8248 - dense_113_loss: 0.8248 - dense_113_MSE: 3.9559 - dense_116_MSE: 12.5601 - val_loss: 0.8014 - val_dense_113_loss: 0.8014 - val_dense_113_MSE: 3.5354 - val_dense_116_MSE: 12.0309
Epoch 88/1200
1556/1561 [============================>.] - ETA: 0s - loss: 0.8232 - dense_113_loss: 0.8232 - dense_113_MSE: 3.9489 - dense_116_MSE: 12.5598
Epoch 00088: saving model to NNmodels/weights.88-0.81217.h5
1561/1561 [==============================] - 17s 11ms/step - loss: 0.8231 - dense_113_loss: 0.8231 - dense_113_MSE: 3.9459 - dense_116_MSE: 12.5642 - val_loss: 0.8122 - val_dense_113_loss: 0.8122 - val_dense_113_MSE: 3.5941 - val_dense_116_MSE: 12.0674
Epoch 89/1200
1558/1561 [============================>.] - ETA

...

Epoch 1129/1200
1559/1561 [============================>.] - ETA: 0s - loss: 0.7990 - dense_113_loss: 0.7990 - dense_113_MSE: 3.6084 - dense_116_MSE: 12.5430
Epoch 01129: saving model to NNmodels/weights.1129-0.79082.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7990 - dense_113_loss: 0.7990 - dense_113_MSE: 3.6075 - dense_116_MSE: 12.5428 - val_loss: 0.7908 - val_dense_113_loss: 0.7908 - val_dense_113_MSE: 3.3621 - val_dense_116_MSE: 12.0187
Epoch 1130/1200
1555/1561 [============================>.] - ETA: 0s - loss: 0.8012 - dense_113_loss: 0.8012 - dense_113_MSE: 3.6179 - dense_116_MSE: 12.5386
Epoch 01130: saving model to NNmodels/weights.1130-0.79476.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.8012 - dense_113_loss: 0.8012 - dense_113_MSE: 3.6151 - dense_116_MSE: 12.5449 - val_loss: 0.7948 - val_dense_113_loss: 0.7948 - val_dense_113_MSE: 3.3905 - val_dense_116_MSE: 12.0896
Epoch 1131/1200
1555/1561 [============================>

Epoch 1147/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.7987 - dense_113_loss: 0.7987 - dense_113_MSE: 3.5952 - dense_116_MSE: 12.5401
Epoch 01147: saving model to NNmodels/weights.1147-0.79413.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7987 - dense_113_loss: 0.7987 - dense_113_MSE: 3.5929 - dense_116_MSE: 12.5419 - val_loss: 0.7941 - val_dense_113_loss: 0.7941 - val_dense_113_MSE: 3.3586 - val_dense_116_MSE: 12.0012
Epoch 1148/1200
1555/1561 [============================>.] - ETA: 0s - loss: 0.7980 - dense_113_loss: 0.7980 - dense_113_MSE: 3.5848 - dense_116_MSE: 12.5346
Epoch 01148: saving model to NNmodels/weights.1148-0.78607.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7981 - dense_113_loss: 0.7981 - dense_113_MSE: 3.5823 - dense_116_MSE: 12.5406 - val_loss: 0.7861 - val_dense_113_loss: 0.7861 - val_dense_113_MSE: 3.4397 - val_dense_116_MSE: 12.0910
Epoch 1149/1200
1556/1561 [============================>

Epoch 1165/1200
1555/1561 [============================>.] - ETA: 0s - loss: 0.7996 - dense_113_loss: 0.7996 - dense_113_MSE: 3.5748 - dense_116_MSE: 12.5283
Epoch 01165: saving model to NNmodels/weights.1165-0.78607.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7996 - dense_113_loss: 0.7996 - dense_113_MSE: 3.5736 - dense_116_MSE: 12.5349 - val_loss: 0.7861 - val_dense_113_loss: 0.7861 - val_dense_113_MSE: 3.3646 - val_dense_116_MSE: 12.0283
Epoch 1166/1200
1557/1561 [============================>.] - ETA: 0s - loss: 0.7996 - dense_113_loss: 0.7996 - dense_113_MSE: 3.5903 - dense_116_MSE: 12.5385
Epoch 01166: saving model to NNmodels/weights.1166-0.79083.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7996 - dense_113_loss: 0.7996 - dense_113_MSE: 3.5882 - dense_116_MSE: 12.5402 - val_loss: 0.7908 - val_dense_113_loss: 0.7908 - val_dense_113_MSE: 3.3983 - val_dense_116_MSE: 12.0621
Epoch 1167/1200
1560/1561 [============================>

1561/1561 [==============================] - 14s 9ms/step - loss: 0.8000 - dense_113_loss: 0.8000 - dense_113_MSE: 3.6028 - dense_116_MSE: 12.5374 - val_loss: 0.7978 - val_dense_113_loss: 0.7978 - val_dense_113_MSE: 3.3796 - val_dense_116_MSE: 12.0642
Epoch 1183/1200
1558/1561 [============================>.] - ETA: 0s - loss: 0.7983 - dense_113_loss: 0.7983 - dense_113_MSE: 3.5997 - dense_116_MSE: 12.5498
Epoch 01183: saving model to NNmodels/weights.1183-0.78729.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7983 - dense_113_loss: 0.7983 - dense_113_MSE: 3.5980 - dense_116_MSE: 12.5491 - val_loss: 0.7873 - val_dense_113_loss: 0.7873 - val_dense_113_MSE: 3.4287 - val_dense_116_MSE: 12.1495
Epoch 1184/1200
1560/1561 [============================>.] - ETA: 0s - loss: 0.8008 - dense_113_loss: 0.8008 - dense_113_MSE: 3.6007 - dense_116_MSE: 12.5471
Epoch 01184: saving model to NNmodels/weights.1184-0.79217.h5
1561/1561 [==============================] - 13s 8ms/ste

Epoch 1200/1200
1559/1561 [============================>.] - ETA: 0s - loss: 0.7989 - dense_113_loss: 0.7989 - dense_113_MSE: 3.5949 - dense_116_MSE: 12.5404
Epoch 01200: saving model to NNmodels/weights.1200-0.78871.h5
1561/1561 [==============================] - 13s 8ms/step - loss: 0.7989 - dense_113_loss: 0.7989 - dense_113_MSE: 3.5936 - dense_116_MSE: 12.5403 - val_loss: 0.7887 - val_dense_113_loss: 0.7887 - val_dense_113_MSE: 3.3624 - val_dense_116_MSE: 12.0867
